# 1. Setup project

### Load and Visualize AOI from a Local Shapefile (GeoPandas + Folium + Earth Engine)

This section of the notebook performs the following steps:

1. **Check for the presence of shapefiles** in the local directory (`../data/aoi/`).
2. **Authenticate and initialize Google Earth Engine (GEE)** to enable access to remote sensing datasets.
3. **Dynamically import a helper script** (`aoi_loader.py`) containing utility functions to load shapefiles as:

   * a `GeoDataFrame` (for local use and visualization),
   * and an `ee.Geometry` object (for Earth Engine processing).
4. **Load the Area of Interest (AOI)** from the shapefile as both:

   * a `GeoDataFrame` (`aoi_gdf`) using GeoPandas, and
   * an Earth Engine geometry (`aoi_ee`) for cloud-based geospatial analysis.
5. **Create an interactive map** using `folium`, centered on the AOI, and add the AOI layer to it.
6. **Display the map directly within the notebook** (and optionally save it as an HTML file for external viewing).

> 💡 This setup allows smooth interoperability between local GIS data and cloud processing via Earth Engine.

---

> ** Note: Earth Engine Authentication**
>
> When you run `ee.Authenticate()` for the first time, a browser window opens asking you to log in to your Google account and authorize Earth Engine access.
> Once authorized, a **credentials token is saved locally** so that you don’t have to log in again every time.
>
> **Where is the token stored?**
>
> * **Windows:**
>   `C:\Users\<YourUsername>\.config\earthengine\credentials`
>
> * **Linux/macOS:**
>   `~/.config/earthengine/credentials`
>
> On subsequent runs, Earth Engine loads the saved credentials automatically.
>
> **To force re-authentication**, you can:
>
> * Delete or rename the `credentials` file manually, or
> * Use:
>
>   ```python
>   ee.Authenticate(clear_credentials=True)
>   ```

In [47]:
import os
import sys
import importlib.util

import ee
import folium
import geemap.foliumap as geemap
import geopandas as gpd

from IPython.display import display

In [48]:

print(os.path.exists("../data/aoi/aoi_1km7x16_3035.shp"))

ee.Authenticate()
ee.Initialize()

True


In [49]:
def load_module(name, rel_path):
    module_path = os.path.abspath(rel_path)
    spec = importlib.util.spec_from_file_location(name, module_path)
    module = importlib.util.module_from_spec(spec)
    sys.modules[name] = module
    spec.loader.exec_module(module)
    return module

sys.path.append(os.path.abspath("../scripts"))

aoi_loader      = load_module("aoi_loader", "../scripts/aoi_loader.py")
download_s1     = load_module("download_s1", "../scripts/download_s1.py")
conv_db_sm     = load_module("conv_db_sm", "../scripts/conv_db_sm.py")




In [50]:
# 1. load AOI shapefile
print(os.path.exists("../data/aoi/aoi_1km7x16_3035.shp"))

# GeoDataFrame AOI (folium)
aoi_gdf = aoi_loader.load_shapefile_as_gdf("../data/aoi/aoi_1km7x16_3035.shp")
print(type(aoi_gdf))

# 📍 Calcola centro per la mappa
center = aoi_gdf.to_crs(epsg=4326).geometry.centroid.iloc[0]
# Converti in coordinate geografiche epsg=4326: lo shapefile è in epsg=3035
print(center)
center_coords = [center.y, center.x]

# 🌍 Crea mappa centrata
m = folium.Map(location=center_coords, zoom_start=12)
folium.GeoJson(aoi_gdf).add_to(m)

m.save("map_aoi.html")
print("💾 Saved 'map_aoi.html' in the current directory")

from IPython.display import display
display(m)


True
<class 'geopandas.geodataframe.GeoDataFrame'>
POINT (12.201048660903087 44.458029809288234)
💾 Saved 'map_aoi.html' in the current directory


C:\Users\Minutella Francesco\AppData\Local\Temp\ipykernel_21240\728124744.py:9: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  center = aoi_gdf.to_crs(epsg=4326).geometry.centroid.iloc[0]


In [51]:
# Earth Engine AOI
aoi_ee = aoi_loader.load_shapefile_as_ee("../data/aoi/aoi_1km7x16_3035.shp")

img = download_s1.get_s1_vv_image(aoi_ee, "2024-04-01", "2024-09-30")

Map = geemap.Map(center=[center_coords[0], center_coords[1]], zoom=12)
Map.addLayer(img, {"min": -25, "max": 0}, "SAR VV (Apr–Sep)")
Map.addLayer(ee.FeatureCollection(aoi_ee), {}, "AOI")
Map


In [52]:
img_clipped = img.clip(aoi_ee)

Map = geemap.Map(center=center_coords, zoom=12)
Map.addLayer(img_clipped, {"min": -25, "max": 0}, "SAR VV Clipped")
Map.addLayer(ee.FeatureCollection(aoi_ee), {}, "AOI")
Map


In [53]:
task = ee.batch.Export.image.toDrive(
    image=img_clipped,
    description='export_sar_vv_clipped',
    folder='earthengine',  # 📁 Google Drive folder
    fileNamePrefix='sarvv2024_04_09',
    scale=10,  # 10m
    region=aoi_ee,
    fileFormat='GeoTIFF'
)

task.start()
print("🚀 Export in GoogleDrive complite: https://code.earthengine.google.com/tasks ")


🚀 Export in GoogleDrive complite: https://code.earthengine.google.com/tasks 


In [54]:
# Coefficienti
a = -4.33
b = -38.3

# Calcolo umidità del suolo
soil_moisture_img = conv_db_sm.vv_to_soil_moisture(img_clipped, a, b)


In [ ]:
Map = geemap.Map(center=center_coords, zoom=12)
Map.addLayer(soil_moisture_img, {"min": 0, "max": 50, "palette": ['#ffffcc','#a1dab4','#41b6c4','#2c7fb8','#253494']}, "Soil Moisture (%)")
#Map.addLayer(ee.FeatureCollection(aoi_ee), {}, "AOI")

legend_dict = {
    '0–10%':  '#ffffcc',
    '10–20%': '#a1dab4',
    '20–30%': '#41b6c4',
    '30–40%': '#2c7fb8',
    '40–50%': '#253494'
}
Map.add_legend(title="Umidità del suolo (%)", legend_dict=legend_dict)
print(legend_dict)

Map



{'0–10%': '#ffffcc', '10–20%': '#a1dab4', '20–30%': '#41b6c4', '30–40%': '#2c7fb8', '40–50%': '#253494'}


In [59]:
# SoilGrids Globali da ISRIC
sand_0_5 = ee.Image("projects/soilgrids-isric/sand_mean").select("sand_0-5cm_mean")
silt_0_5 = ee.Image("projects/soilgrids-isric/silt_mean").select("silt_0-5cm_mean")
clay_0_5 = ee.Image("projects/soilgrids-isric/clay_mean").select("clay_0-5cm_mean")

# Clip per AOI
sand_clipped = sand_0_5.clip(aoi_ee)
silt_clipped = silt_0_5.clip(aoi_ee)
clay_clipped = clay_0_5.clip(aoi_ee)

# Esporta su Drive
task = ee.batch.Export.image.toDrive(
    image = sand_clipped,
    description = 'sand_0_5',
    folder = 'soilgrids',
    region = aoi_ee,
    scale = 250,
    maxPixels = 1e13
)
task.start()
print("🚀 Export avviato su Google Drive: https://code.earthengine.google.com/tasks")


🚀 Export avviato su Google Drive: https://code.earthengine.google.com/tasks


In [ ]:
# Visualizzazione
Map = geemap.Map(center=center_coords, zoom=12)
Map.addLayer(sand_clipped, {
    "min": 0, "max": 100,
    "palette": ['#ffffcc','#c2e699','#78c679','#31a354','#006837']
}, "Sabbia 0–5 cm")
Map.addLayer(ee.FeatureCollection(aoi_ee), {}, "AOI")
Map


SyntaxError: invalid syntax. Perhaps you forgot a comma? (1291554160.py, line 3)